In [1]:
import sys
sys.path.append("../..")

In [2]:
from encoding.parse_rooms import parse_rooms, parse_exits, dictify_rooms

In [3]:
rooms = parse_rooms("../../encoding/dsl/rooms_for_alloy.txt")
exits = parse_exits("../../encoding/dsl/exits_for_alloy.txt")
design = dictify_rooms(rooms, exits)

In [4]:
from bdds.bdd_core import *
from bdds.node_bdds import *

In [5]:
all_nodes = []
for r, room in rooms.items():
    for node in room.graph.name_node.keys():
        all_nodes.append(node)
node_ids = {n:i for i,n in enumerate(all_nodes)}

In [6]:
node_id_rev = {i:n for n,i in node_ids.items()}

In [7]:
context = mk_context_id(node_ids)

In [8]:
trans_important = context.bdd.load("../../output/important_closure.dddmp")[0]

In [9]:
#from omega.symbolic.fol import Context as OmegaContext

In [10]:
item_vars = prevs[1:]

In [11]:
nodes = context.exist(item_vars + nexts, trans_important)

In [12]:
important_nodes = [node_id_rev[i["node_id_prev"]] for i in context.pick_iter(nodes)]

In [13]:
node_vars_prev = { f"at_{i}_prev" : (0,1) for i in important_nodes }
node_vars_next = { f"at_{i}_next" : (0,1) for i in important_nodes }

In [14]:
context.declare(**node_vars_prev)
context.declare(**node_vars_next)

In [15]:
def static_order_score(name):
  var,tense,bit = name.rsplit('_', 2)
  return (-int(var=="node_id"),-int(var.startswith("at")),-int(bit),var,tense)
context.bdd.reorder({v:i for i,v in enumerate(sorted(context.bdd.vars, key=static_order_score))})

In [16]:
binary_to_one_hot = context.true
for i in important_nodes:
    nid = node_ids[i]
    binary_to_one_hot &= context.add_expr(f"node_id_prev = {nid} <-> at_{i}_prev = 1")
    binary_to_one_hot &= context.add_expr(f"node_id_next = {nid} <-> at_{i}_next = 1")

In [17]:
(trans_important & binary_to_one_hot).dag_size, trans_important.dag_size

(337284, 289590)

In [18]:
trans_= context.exist(["node_id_prev", "node_id_next"], trans_important & binary_to_one_hot)

In [19]:
trans_one_hot.dag_size

297968

In [ ]:
# Uniqueness